In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import json
import osmnx as ox
import networkx as nx
import osmium as o

# Datos - Latitud, Longitud - Municipios

In [3]:
df_municipios_ubicacion = pd.read_csv('../../Data/DIVIPOLA_Municipios.csv')

In [4]:
df_municipios_ubicacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121 entries, 0 to 1120
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Código_Dep    1121 non-null   int64  
 1   Departamento  1121 non-null   object 
 2   Código_Mun    1121 non-null   int64  
 3   Municipio     1121 non-null   object 
 4   Tipo          1121 non-null   object 
 5   Latitud       1121 non-null   float64
 6   Longitud      1121 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 61.4+ KB


In [5]:
df_municipios_ubicacion.head()

,Código_Dep,Departamento,Código_Mun,Municipio,Tipo,Latitud,Longitud
0,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,6.257590,-75.611031
1,5,ANTIOQUIA,5002,ABEJORRAL,Municipio,5.803728,-75.438474
2,5,ANTIOQUIA,5004,ABRIAQUÍ,Municipio,6.627569,-76.085978
3,5,ANTIOQUIA,5021,ALEJANDRÍA,Municipio,6.365534,-75.090597
4,5,ANTIOQUIA,5030,AMAGÁ,Municipio,6.032922,-75.708003


# Datos - REPS

In [6]:
df_reps = pd.read_csv('../../Data/REPS.csv')

C:\Users\SGM\AppData\Local\Temp\ipykernel_14712\3122532733.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reps = pd.read_csv('../../Data/REPS.csv')


In [7]:
df_reps['codigo_prestador'] = df_reps['CodigoPrestador']
df_reps['codigo_habilitacion_sede'] = df_reps['CodigoHabilitacionSede']
df_reps.iloc[56503]['CodigoPrestador'], df_reps.iloc[56503]['CodigoHabilitacionSede'], df_reps.iloc[74472]['CodigoPrestador'], df_reps.iloc[74472]['CodigoHabilitacionSede']

df_reps.at[56503, 'codigo_prestador'] = df_reps.iloc[56503]['CodigoPrestador'].replace(" ", "")
df_reps.at[56503, 'codigo_habilitacion_sede'] = df_reps.iloc[56503]['CodigoHabilitacionSede'].replace(" ", "")
df_reps.at[74472, 'codigo_prestador'] = df_reps.iloc[74472]['CodigoPrestador'].replace(" ", "")
df_reps.at[74472, 'codigo_habilitacion_sede'] = df_reps.iloc[74472]['CodigoHabilitacionSede'].replace(" ", "")

df_reps['codigo_prestador'] = df_reps['codigo_prestador'].astype(np.int64)
df_reps['codigo_habilitacion_sede'] = df_reps['codigo_habilitacion_sede'].astype(np.int64)

In [8]:
departamentos = [
    'Amazonas', 'Antioquia', 'Arauca', 'Atlántico',
    'Bogotá D.C', 'Bolívar', 'Boyacá', 'Caldas',
    'Caquetá', 'Casanare', 'Cauca', 'Cesar', 
    'Chocó', 'Córdoba', 'Cundinamarca', 'Guainía', 
    'Guaviare', 'Huila', 'La Guajira', 'Magdalena',
    'Meta', 'Nariño', 'Norte de Santander', 'Putumayo',
    'Quindío', 'Risaralda', 'San Andrés y Providencia', 'Santander',
    'Sucre', 'Tolima', 'Valle del cauca', 'Vaupés', 
    'Vichada'
    ]

distritos_especiales = [
    'Barranquilla',
    'Buenaventura',
    'Cartagena',
    'Santa Marta'
]

distritos_especiales_departamentos = {
    'Barranquilla': 'Atlántico',
    'Buenaventura': 'Valle del cauca',
    'Cartagena': 'Bolívar',
    'Santa Marta': 'Magdalena',
    'Cali': 'Valle del cauca'
}

In [9]:
df_reps['Departamento'] = df_reps['DepartamentoSedeDesc'].replace(distritos_especiales_departamentos)
df_reps = df_reps[df_reps['ClasePrestadorDesc'] == 'Instituciones Prestadoras de Servicios de Salud - IPS']

In [10]:
columnas = [
    'DepartamentoSedeDesc',
    'MunicipioPrestadorDesc',
    'MunicipioSede',
    'MunicipioSedeDesc',
    'codigo_prestador',
    'codigo_habilitacion_sede',
    'NaturalezaJuridica',
    # 'ClasePrestadorDesc'
]

In [11]:
df_reps = df_reps[columnas]

In [12]:
df_reps.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19726 entries, 0 to 76393
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   DepartamentoSedeDesc      19726 non-null  object
 1   MunicipioPrestadorDesc    19726 non-null  object
 2   MunicipioSede             19726 non-null  int64 
 3   MunicipioSedeDesc         19726 non-null  object
 4   codigo_prestador          19726 non-null  int64 
 5   codigo_habilitacion_sede  19726 non-null  int64 
 6   NaturalezaJuridica        19726 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.2+ MB


In [13]:
df_reps.head()

,DepartamentoSedeDesc,MunicipioPrestadorDesc,MunicipioSede,MunicipioSedeDesc,codigo_prestador,codigo_habilitacion_sede,NaturalezaJuridica
0,Amazonas,LETICIA,91263,EL ENCANTO,9100100019,912630001911,Pública
1,Amazonas,LETICIA,91405,LA CHORRERA,9100100019,914050001914,Pública
2,Amazonas,LETICIA,91407,LA PEDRERA,9100100019,914070001915,Pública
3,Amazonas,LETICIA,91430,LA VICTORIA,9100100019,914300001916,Pública
5,Amazonas,LETICIA,91001,LETICIA,9100100019,910010001901,Pública


# Datos - IPS Complejidad

In [14]:
df_ips = pd.read_csv("../../Data/IPS_Complejidad.csv")

In [15]:
departamentos = [
    'Amazonas', 'Antioquia', 'Arauca', 'Atlántico',
    'Bogotá D.C', 'Bolívar', 'Boyacá', 'Caldas',
    'Caquetá', 'Casanare', 'Cauca', 'Cesar', 
    'Chocó', 'Córdoba', 'Cundinamarca', 'Guainía', 
    'Guaviare', 'Huila', 'La Guajira', 'Magdalena',
    'Meta', 'Nariño', 'Norte de Santander', 'Putumayo',
    'Quindío', 'Risaralda', 'San Andrés y Providencia', 'Santander',
    'Sucre', 'Tolima', 'Valle del cauca', 'Vaupés', 
    'Vichada'
    ]

distritos_especiales = [
    'Barranquilla',
    'Buenaventura',
    'Cartagena',
    'Santa Marta'
]

distritos_especiales_departamentos = {
    'Barranquilla': 'Atlántico',
    'Buenaventura': 'Valle del cauca',
    'Cartagena': 'Bolívar',
    'Santa Marta': 'Magdalena'
}

In [16]:
columnas = [
    'depa_nombre',
    'muni_nombre',
    'codigo_habilitacion',
    # 'razon_social',
    'nivel',
    # 'habilitado',
    # 'naju_codigo',
    'naju_nombre',
    'clase_persona'
]

In [17]:
df_ips = df_ips[columnas]
df_ips['Departamento'] = df_ips['depa_nombre'].replace(distritos_especiales_departamentos)

In [18]:
df_ips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11466 entries, 0 to 11465
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   depa_nombre          11466 non-null  object 
 1   muni_nombre          11466 non-null  object 
 2   codigo_habilitacion  11466 non-null  int64  
 3   nivel                1002 non-null   float64
 4   naju_nombre          11466 non-null  object 
 5   clase_persona        11466 non-null  object 
 6   Departamento         11466 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 627.2+ KB


In [19]:
df_ips.head()

,depa_nombre,muni_nombre,codigo_habilitacion,nivel,naju_nombre,clase_persona,Departamento
0,Amazonas,LETICIA,9100100019,2.0,Pública,JURIDICO,Amazonas
1,Amazonas,LETICIA,9100100185,NaN,Privada,JURIDICO,Amazonas
2,Amazonas,LETICIA,9100100069,NaN,Privada,JURIDICO,Amazonas
3,Amazonas,LETICIA,9100100158,NaN,Privada,JURIDICO,Amazonas
4,Amazonas,LETICIA,9100100144,NaN,Privada,JURIDICO,Amazonas


# IPS Públicas

In [20]:
df_ips_publica = df_ips[df_ips['naju_nombre'] == 'Pública']

In [21]:
df_ips_publica

,depa_nombre,muni_nombre,codigo_habilitacion,nivel,naju_nombre,clase_persona,Departamento
0,Amazonas,LETICIA,9100100019,2.0,Pública,JURIDICO,Amazonas
6,Amazonas,LETICIA,9100100168,1.0,Pública,JURIDICO,Amazonas
14,Antioquia,ABEJORRAL,500204360,1.0,Pública,JURIDICO,Antioquia
15,Antioquia,ALEJANDRÍA,502105098,1.0,Pública,JURIDICO,Antioquia
16,Antioquia,AMAGÁ,503004374,1.0,Pública,JURIDICO,Antioquia
...,...,...,...,...,...,...,...
11447,Valle del cauca,YUMBO,7689204073,1.0,Pública,JURIDICO,Valle del cauca
11453,Valle del cauca,ZARZAL,7689504656,2.0,Pública,JURIDICO,Valle del cauca
11455,Vaupés,MITÚ,9700100013,1.0,Pública,JURIDICO,Vaupés
11456,Vaupés,MITÚ,9700100001,2.0,Pública,JURIDICO,Vaupés


In [22]:
df_reps_publica = df_reps[df_reps['NaturalezaJuridica'] == 'Pública']

In [23]:
df_reps_publica

,DepartamentoSedeDesc,MunicipioPrestadorDesc,MunicipioSede,MunicipioSedeDesc,codigo_prestador,codigo_habilitacion_sede,NaturalezaJuridica
0,Amazonas,LETICIA,91263,EL ENCANTO,9100100019,912630001911,Pública
1,Amazonas,LETICIA,91405,LA CHORRERA,9100100019,914050001914,Pública
2,Amazonas,LETICIA,91407,LA PEDRERA,9100100019,914070001915,Pública
3,Amazonas,LETICIA,91430,LA VICTORIA,9100100019,914300001916,Pública
5,Amazonas,LETICIA,91001,LETICIA,9100100019,910010001901,Pública
...,...,...,...,...,...,...,...
76359,Vaupés,MITÚ,97889,YAVARATÉ,9700100001,978890000166,Pública
76360,Vichada,PUERTO CARREÑO,99773,CUMARIBO,9900100006,997730000607,Pública
76365,Vichada,PUERTO CARREÑO,99524,LA PRIMAVERA,9900100006,995240000608,Pública
76370,Vichada,PUERTO CARREÑO,99001,PUERTO CARREÑO,9900100006,990010000601,Pública


In [24]:
set1 = set(df_ips_publica['codigo_habilitacion'].unique())
len(set1)

1002

In [25]:
set2 = set(df_reps_publica['codigo_prestador'].unique())
len(set2)

999

In [26]:
len(set.intersection(set1, set2))

996

# Merge

In [27]:
df_reps_merge = pd.merge(df_reps_publica, df_ips_publica, how='inner', left_on='codigo_prestador', right_on='codigo_habilitacion')

In [28]:
df_reps_merge

,DepartamentoSedeDesc,MunicipioPrestadorDesc,MunicipioSede,MunicipioSedeDesc,codigo_prestador,codigo_habilitacion_sede,NaturalezaJuridica,depa_nombre,muni_nombre,codigo_habilitacion,nivel,naju_nombre,clase_persona,Departamento
0,Amazonas,LETICIA,91263,EL ENCANTO,9100100019,912630001911,Pública,Amazonas,LETICIA,9100100019,2.0,Pública,JURIDICO,Amazonas
1,Amazonas,LETICIA,91405,LA CHORRERA,9100100019,914050001914,Pública,Amazonas,LETICIA,9100100019,2.0,Pública,JURIDICO,Amazonas
2,Amazonas,LETICIA,91407,LA PEDRERA,9100100019,914070001915,Pública,Amazonas,LETICIA,9100100019,2.0,Pública,JURIDICO,Amazonas
3,Amazonas,LETICIA,91430,LA VICTORIA,9100100019,914300001916,Pública,Amazonas,LETICIA,9100100019,2.0,Pública,JURIDICO,Amazonas
4,Amazonas,LETICIA,91001,LETICIA,9100100019,910010001901,Pública,Amazonas,LETICIA,9100100019,2.0,Pública,JURIDICO,Amazonas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3608,Vaupés,MITÚ,97889,YAVARATÉ,9700100001,978890000166,Pública,Vaupés,MITÚ,9700100001,2.0,Pública,JURIDICO,Vaupés
3609,Vichada,PUERTO CARREÑO,99773,CUMARIBO,9900100006,997730000607,Pública,Vichada,PUERTO CARREÑO,9900100006,2.0,Pública,JURIDICO,Vichada
3610,Vichada,PUERTO CARREÑO,99524,LA PRIMAVERA,9900100006,995240000608,Pública,Vichada,PUERTO CARREÑO,9900100006,2.0,Pública,JURIDICO,Vichada
3611,Vichada,PUERTO CARREÑO,99001,PUERTO CARREÑO,9900100006,990010000601,Pública,Vichada,PUERTO CARREÑO,9900100006,2.0,Pública,JURIDICO,Vichada


In [29]:
# Nulos sin nivel
df_reps_merge[df_reps_merge.isna().any(axis=1)]

,DepartamentoSedeDesc,MunicipioPrestadorDesc,MunicipioSede,MunicipioSedeDesc,codigo_prestador,codigo_habilitacion_sede,NaturalezaJuridica,depa_nombre,muni_nombre,codigo_habilitacion,nivel,naju_nombre,clase_persona,Departamento
1256,Cauca,INZÁ,19355,INZÁ,1935508300,193550830001,Pública,Cauca,INZÁ,1935508300,NaN,Pública,JURIDICO,Cauca
1301,Cauca,POPAYÁN,19001,POPAYÁN,1900108183,190010818301,Pública,Cauca,POPAYÁN,1900108183,NaN,Pública,JURIDICO,Cauca
1302,Cauca,POPAYÁN,19001,POPAYÁN,1900108183,190010818313,Pública,Cauca,POPAYÁN,1900108183,NaN,Pública,JURIDICO,Cauca
3231,Sucre,SINCELEJO,70001,SINCELEJO,7000100504,700010050401,Pública,Sucre,SINCELEJO,7000100504,NaN,Pública,JURIDICO,Sucre
3246,Tolima,IBAGUÉ,73168,CHAPARRAL,7300100855,731680085507,Pública,Tolima,IBAGUÉ,7300100855,NaN,Pública,JURIDICO,Tolima
3247,Tolima,IBAGUÉ,73217,COYAIMA,7300100855,732170085502,Pública,Tolima,IBAGUÉ,7300100855,NaN,Pública,JURIDICO,Tolima
3248,Tolima,IBAGUÉ,73001,IBAGUÉ,7300100855,730010085501,Pública,Tolima,IBAGUÉ,7300100855,NaN,Pública,JURIDICO,Tolima
3249,Tolima,IBAGUÉ,73483,NATAGAIMA,7300100855,734830085504,Pública,Tolima,IBAGUÉ,7300100855,NaN,Pública,JURIDICO,Tolima
3250,Tolima,IBAGUÉ,73504,ORTEGA,7300100855,735040085506,Pública,Tolima,IBAGUÉ,7300100855,NaN,Pública,JURIDICO,Tolima
3251,Tolima,IBAGUÉ,73504,ORTEGA,7300100855,735040085508,Pública,Tolima,IBAGUÉ,7300100855,NaN,Pública,JURIDICO,Tolima


In [30]:
df_ips_publica[df_ips_publica.isna().any(axis=1)]

,depa_nombre,muni_nombre,codigo_habilitacion,nivel,naju_nombre,clase_persona,Departamento
5046,Cauca,INZÁ,1935508300,NaN,Pública,JURIDICO,Cauca
5080,Cauca,POPAYÁN,1900108183,NaN,Pública,JURIDICO,Cauca
10186,Sucre,SINCELEJO,7000100504,NaN,Pública,JURIDICO,Sucre
10502,Tolima,IBAGUÉ,7300100855,NaN,Pública,JURIDICO,Tolima


In [31]:
df_reps_merge.dropna(inplace=True)
df_reps_merge['nivel'] = df_reps_merge['nivel'].astype('int64')

In [32]:
df_reps_merge = df_reps_merge[['Departamento', 'MunicipioSede', 'MunicipioSedeDesc', 'nivel', 'codigo_habilitacion_sede']]

In [33]:
df_reps_merge

,Departamento,MunicipioSede,MunicipioSedeDesc,nivel,codigo_habilitacion_sede
0,Amazonas,91263,EL ENCANTO,2,912630001911
1,Amazonas,91405,LA CHORRERA,2,914050001914
2,Amazonas,91407,LA PEDRERA,2,914070001915
3,Amazonas,91430,LA VICTORIA,2,914300001916
4,Amazonas,91001,LETICIA,2,910010001901
...,...,...,...,...,...
3608,Vaupés,97889,YAVARATÉ,2,978890000166
3609,Vichada,99773,CUMARIBO,2,997730000607
3610,Vichada,99524,LA PRIMAVERA,2,995240000608
3611,Vichada,99001,PUERTO CARREÑO,2,990010000601


In [34]:
df_reps_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3602 entries, 0 to 3612
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Departamento              3602 non-null   object
 1   MunicipioSede             3602 non-null   int64 
 2   MunicipioSedeDesc         3602 non-null   object
 3   nivel                     3602 non-null   int64 
 4   codigo_habilitacion_sede  3602 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 168.8+ KB


In [35]:
df_reps_merge['nivel'].unique()

array([2, 1, 3], dtype=int64)

In [36]:
df_reps_merge = pd.merge(df_reps_merge, df_municipios_codigos[['Codigo_Municipio', 'Municipio']], how='inner', left_on='MunicipioSede', right_on='Codigo_Municipio')
df_reps_merge = pd.merge(df_reps_merge, df_departamentos_codigos[['Codigo', 'Departamento']], how='left', left_on='Departamento', right_on='Departamento')
df_reps_merge['Municipio_Departamento'] = df_reps_merge['Municipio'] + " (" + df_reps_merge['Departamento'] + ")"

NameError: name 'df_municipios_codigos' is not defined

In [ ]:
df_reps_merge = df_reps_merge[['Departamento', 'Codigo', 'Municipio', 'Codigo_Municipio', 'Municipio_Departamento', 'codigo_habilitacion_sede', 'nivel']]

In [ ]:
df_reps_merge

,Departamento,Codigo,Municipio,Codigo_Municipio,Municipio_Departamento,codigo_habilitacion_sede,nivel
0,Amazonas,91,El Encanto,91263,El Encanto (Amazonas),912630001911,2
1,Amazonas,91,La Chorrera,91405,La Chorrera (Amazonas),914050001914,2
2,Amazonas,91,La Pedrera,91407,La Pedrera (Amazonas),914070001915,2
3,Amazonas,91,La Victoria,91430,La Victoria (Amazonas),914300001916,2
4,Amazonas,91,Leticia,91001,Leticia (Amazonas),910010001901,2
...,...,...,...,...,...,...,...
3597,Vaupés,97,Yavaraté,97889,Yavaraté (Vaupés),978890000166,2
3598,Vichada,99,Cumaribo,99773,Cumaribo (Vichada),997730000607,2
3599,Vichada,99,La Primavera,99524,La Primavera (Vichada),995240000608,2
3600,Vichada,99,Puerto Carreño,99001,Puerto Carreño (Vichada),990010000601,2


# Cálculo Distancias

In [ ]:
df_municipios_ubicacion

,Código_Dep,Departamento,Código_Mun,Municipio,Tipo,Latitud,Longitud
0,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,6.257590,-75.611031
1,5,ANTIOQUIA,5002,ABEJORRAL,Municipio,5.803728,-75.438474
2,5,ANTIOQUIA,5004,ABRIAQUÍ,Municipio,6.627569,-76.085978
3,5,ANTIOQUIA,5021,ALEJANDRÍA,Municipio,6.365534,-75.090597
4,5,ANTIOQUIA,5030,AMAGÁ,Municipio,6.032922,-75.708003
...,...,...,...,...,...,...,...
1116,97,VAUPÉS,97889,YAVARATÉ,Área no municipalizada,0.833312,-69.618678
1117,99,VICHADA,99001,PUERTO CARREÑO,Municipio,5.836530,-68.141222
1118,99,VICHADA,99524,LA PRIMAVERA,Municipio,5.517594,-69.620441
1119,99,VICHADA,99624,SANTA ROSALÍA,Municipio,4.968581,-70.659971


In [ ]:
med = df_municipios_ubicacion.iloc[0]
med_lat = med['Latitud']
med_lon = med['Longitud']
med_lat, med_lon

bog = df_municipios_ubicacion.iloc[148]
bog_lat = bog['Latitud']
bog_lon = bog['Longitud']

(med_lat, med_lon), (bog_lat, bog_lon)

((6.257590259, -75.61103107), (4.316107698, -74.1810727))

In [37]:
# graph = ox.graph_from_place('Colombia', network_type='drive', simplify=True)

In [39]:
graph = ox.graph_from_xml('../../Data/colombia-latest.osm.bz2')

KeyboardInterrupt: 

In [ ]:
def calcular_distancia_osmnx(G, mun1, mun2):    
    try:
        orig_node = ox.distance.nearest_nodes(G, G.nodes[mun1['']]['y'], G.nodes[codigo_mun1]['x'])
        dest_node = ox.distance.nearest_nodes(G, G.nodes[codigo_mun2]['y'], G.nodes[codigo_mun2]['x'])
        route = nx.shortest_path(G, orig_node, dest_node, weight='length')
        distance = sum(G.edges[u, v, 0]['length'] for u, v in zip(route[:-1], route[1:])) / 1000  # Convertir a kilómetros
        return distance
    except Exception as e:
        print(f"Error al calcular la distancia entre {codigo_mun1} y {codigo_mun2}: {e}")
        return None

In [ ]:
orig_node = ox.distance.nearest_nodes(graph, bog['Latitud'], bog['Longitud'])
dest_node = ox.distance.nearest_nodes(graph, med['Latitud'], med['Longitud'])

In [ ]:
route = nx.shortest_path(graph, orig_node, dest_node, weight='length')

In [ ]:
route_length_km = sum(graph.edges[u, v, 0]['length'] for u, v in zip(route[:-1], route[1:])) / 1000